Carico il file csv

In [51]:
import pandas as pd
import os
import re

In [54]:
# Ottieni la directory di lavoro corrente
script_dir = os.getcwd()

# Costruisci il percorso relativo al file CSV
file_path = os.path.join(script_dir, "Data", "main_data", "actors.csv")

# Costruisci il percorso relativo al file CSV pulito
clean_path = os.path.join(script_dir, "Data", "clean_data")

# Caricare il CSV in un DataFrame
df = pd.read_csv(file_path)

# Visualizzare le prime righe
df.head()

,id,name,role
0,1000001,Margot Robbie,Barbie
1,1000001,Ryan Gosling,Ken
2,1000001,America Ferrera,Gloria
3,1000001,Ariana Greenblatt,Sasha
4,1000001,Issa Rae,Barbie


In [59]:
# Rimuove valori nulli se necessario
df.dropna(inplace=True)

# Rimuove duplicati
df.drop_duplicates(inplace=True)

In [56]:
# Prende solo la prima parte prima di "/"
df['role'] = df['role'].str.split('/').str[0]

# Rimuove sinonimi tra parentesi
df['role'] = df['role'].apply(lambda x: re.sub(r"\(.*?\)|（.*?）", "", x))

# Standardizza il testo (se ci sono nomi di attori, ruoli, ecc.)
df['name'] = df['name'].str.strip().str.title()
df['role'] = df['role'].str.strip().str.lower()

print(df['role'].unique())


['barbie' 'ken' 'gloria' ... 'an du' 'ba cai' 'zuo yila']


In [58]:
# Rimuove righe con troppi valori nulli (ad esempio, più del 50% dei dati mancanti)
df.dropna(thresh=df.shape[1] * 0.5, inplace=True)

# Sostituisci i valori nulli con "Unknown" o il valore più comune
df['role'] = df['role'].fillna("Unknown")
df['name'] = df['name'].fillna("Unknown")

In [65]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "actors_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")